In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from random import randint
import seaborn as sns
import sys
# from tensorflow.keras.utils import plot_model

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import src.false_labels_effect.util as util
# import src.false_labels_effect.models as models

In [ ]:
# set training and test img path

train_img_path = Path("../data/Images_4c_Poly/Train")
test_img_path = Path("../data/Images_4c_Poly/Test")

# set label path
train_label_path = Path("../data/Labels_4c_Poly/Train.npy")
test_label_path = Path("../data/Labels_4c_Poly/Test.npy")

# load labels dict
train_labels_dict = util.load_labels(train_label_path)
test_labels_dict = util.load_labels(test_label_path)

In [ ]:
# show random example train label
train_exm = "Train_" + str(randint(1, len(train_labels_dict.keys()))).zfill(5)
print(f"Example labels_dict['{train_exm}'] = {train_labels_dict[train_exm]}")

# show random example test label
test_exm = "Test_" + str(randint(1, len(test_labels_dict.keys()))).zfill(5)
print(f"Example labels_dict['{test_exm}'] = {test_labels_dict[test_exm]}")

In [ ]:
# get all main classes
main_classes_list = []
for label_set in [train_labels_dict.values(), test_labels_dict.values()]:
    for label in label_set:
        main_classes_list.append(label['Class'])

main_classes_set = sorted(set(main_classes_list))
main_classes_set

In [ ]:
# get all sub classes
sub_classes_list = []
for label_set in [train_labels_dict.values(), test_labels_dict.values()]:
    for label in label_set:
        sub_classes_list.append(label['Class'] + ' - ' + str(label['Subclass']))
        
sub_classes_set = sorted(set(sub_classes_list))
sub_classes_set

In [ ]:
for label in sub_classes_set:
    print(f'{sub_classes_list.count(label)} - {label}')
    
print('\n---\n')

for label in main_classes_set:
    print(f'{main_classes_list.count(label)} - {label}')

In [ ]:
colorblind_palette = sns.color_palette('colorblind', 10)
colorblind_palette

In [ ]:
def update_label(label: str) -> str:
    label = label.replace('_', ' ').replace(',', ' ').replace("'", '')
    label = label.replace('Green', 'G')
    label = label.replace('Blue', 'B')
    label = label.replace('Red', 'R')
    
    return label

f, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (10, 5))

sns.histplot(sorted(main_classes_list), ax = axes[0])
sns.histplot(sorted(sub_classes_list), ax = axes[1])

plt.subplots_adjust(wspace=0.25, bottom=0.4)
plt.draw()

sns.set(font_scale = 1.2)
sns.despine(left=True, bottom=True)
sns.set_style("whitegrid")

for ind, ax in enumerate(axes):
    
    texts = [update_label(t.get_text())  for t in ax.get_xticklabels()]
    ax.set_xticklabels(texts, rotation = 45, ha = 'right')
    if ind == 0:
        ax.set_ylabel('n')
    else:
        ax.set_ylabel('')
    ax.grid(visible=False, axis='x')
    
axes[0].set_title('Main classes distribution')
axes[1].set_title('Subclasses distribution')

f.savefig('../assets/class_hist.png', dpi=300, facecolor='white')

In [ ]:
# bCNN_4cl = models.create_cnn_model(img_shape=(244, 244), n_classes=4, false_labels_ratio=0)
# plot_model(bCNN_4cl, to_file=f'../assets/basic_CNN_4classes.png', show_shapes=True, show_layer_names=True)

# bCNN_14cl = models.create_cnn_model(img_shape=(244, 244), n_classes=14, false_labels_ratio=0)
# plot_model(bCNN_14cl, to_file=f'../assets/basic_CNN_14classes.png', show_shapes=True, show_layer_names=True)

# resnet_4cl = models.create_resnet_model(img_shape=(244, 244), n_classes=4, false_labels_ratio=0)
# plot_model(resnet_4cl, to_file=f'../assets/resnet_CNN_4classes.png', show_shapes=True, show_layer_names=True)

# resnet_14cl = models.create_resnet_model(img_shape=(244, 244), n_classes=14, false_labels_ratio=0)
# plot_model(resnet_14cl, to_file=f'../assets/resnet_CNN_4classes.png', show_shapes=True, show_layer_names=True)